In [1]:
import sys
sys.path.append("../vit")
import json
from PIL import Image
import torch
from torchvision import transforms
from FeatureSelect_contrast import FeatureSelectModule
from tqdm import tqdm
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from PIL import Image
import torch
from torchvision import transforms
import pandas as pd
import cv2
from skimage import transform
import numpy as np
from torch import nn

In [2]:
from torch.utils.data import DataLoader
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import roc_auc_score
# 设置随机种子
import random
seed = 400
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

In [3]:
class NIH(torch.utils.data.Dataset):
    def __init__(self, data=None, phase = 'train', img_size=(224, 224)):
        self.PRET_MEANS = [0.485, 0.456, 0.406]
        self.PRET_STDS = [0.229, 0.224, 0.225]
        self.img_size = img_size
        self.datas = data
        self.origin_transform = transforms.Compose([
                                                        transforms.Resize(self.img_size),
                                                        transforms.ToTensor(),
                                                        transforms.ConvertImageDtype(torch.float32),
                                                        transforms.Normalize(mean=self.PRET_MEANS, std=self.PRET_STDS)
                                                    ])
                                                    
        ### 图片增强
        if phase == 'train':
            self.transform = transforms.Compose([
                                                    transforms.Resize(self.img_size),
                                                    transforms.RandomAdjustSharpness(2, 0.8),
                                                    transforms.ToTensor(),
                                                    transforms.ConvertImageDtype(torch.float32),
                                                    transforms.Normalize(mean=self.PRET_MEANS, std=self.PRET_STDS)
                                                ])

 
        else:
            self.transform = transforms.Compose([
                                                    transforms.Resize(self.img_size),
                                                    transforms.ToTensor(),
                                                    transforms.ConvertImageDtype(torch.float32),
                                                    transforms.Normalize(mean=self.PRET_MEANS, std=self.PRET_STDS)
                                                ])
    def __len__(self):
        return len(self.datas)

    def __getitem__(self, index):
        fpath = self.datas.iloc[index]['full_path']
        image = Image.open(fpath).convert('RGB')
        origin_image =  self.origin_transform(image)
        aug_image = self.transform(image)
        label = self.datas.iloc[index]['target_vector']
        label = label.astype(np.int64)
        label = torch.from_numpy(label).float()
        return (aug_image, origin_image), label

In [4]:
data_root = '../dataset/NIH-CXR/'
data = pd.read_csv("../dataset/NIH-CXR/Data_Entry_2017_v2020.csv", encoding='GBK')
dummy_labels = ['No Finding', 'Atelectasis', 'Consolidation', 'Infiltration', 'Pneumothorax', 'Edema', 'Emphysema', 'Fibrosis', 'Effusion', 'Pneumonia', 'Pleural_Thickening', 
'Cardiomegaly', 'Nodule', 'Mass', 'Hernia'] # taken from paper

# One Hot Encoding of Finding Labels to dummy_labels
for label in dummy_labels:
    data[label] = data['Finding Labels'].map(lambda result: 1.0 if label in result else 0)
data['full_path'] = data_root + 'all_images/' + data['Image Index'] 
data.head(10)  

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,...,Emphysema,Fibrosis,Effusion,Pneumonia,Pleural_Thickening,Cardiomegaly,Nodule,Mass,Hernia,full_path
0,00000001_000.png,Cardiomegaly,0,1,57,M,PA,2682,2749,0.143,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,../dataset/NIH-CXR/all_images/00000001_000.png
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,../dataset/NIH-CXR/all_images/00000001_001.png
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,../dataset/NIH-CXR/all_images/00000001_002.png
3,00000002_000.png,No Finding,0,2,80,M,PA,2500,2048,0.171,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,../dataset/NIH-CXR/all_images/00000002_000.png
4,00000003_001.png,Hernia,0,3,74,F,PA,2500,2048,0.168,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,../dataset/NIH-CXR/all_images/00000003_001.png
5,00000003_002.png,Hernia,1,3,75,F,PA,2048,2500,0.168,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,../dataset/NIH-CXR/all_images/00000003_002.png
6,00000003_003.png,Hernia|Infiltration,2,3,76,F,PA,2698,2991,0.143,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,../dataset/NIH-CXR/all_images/00000003_003.png
7,00000003_004.png,Hernia,3,3,77,F,PA,2500,2048,0.168,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,../dataset/NIH-CXR/all_images/00000003_004.png
8,00000003_005.png,Hernia,4,3,78,F,PA,2686,2991,0.143,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,../dataset/NIH-CXR/all_images/00000003_005.png
9,00000003_006.png,Hernia,5,3,79,F,PA,2992,2991,0.143,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,../dataset/NIH-CXR/all_images/00000003_006.png


In [5]:
import matplotlib.pyplot as plt # basic plotting
clean_labels = data[dummy_labels].sum().sort_values(ascending= False) # get sorted value_count for clean labels
print(clean_labels) # view tabular results

No Finding            60361.0
Infiltration          19894.0
Effusion              13317.0
Atelectasis           11559.0
Nodule                 6331.0
Mass                   5782.0
Pneumothorax           5302.0
Consolidation          4667.0
Pleural_Thickening     3385.0
Cardiomegaly           2776.0
Emphysema              2516.0
Edema                  2303.0
Fibrosis               1686.0
Pneumonia              1431.0
Hernia                  227.0
dtype: float64


In [17]:
data['target_vector'] = data.apply(lambda target: [target[dummy_labels].values], 1).map(lambda target: target[0])
data.head(5)

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,...,Fibrosis,Effusion,Pneumonia,Pleural_Thickening,Cardiomegaly,Nodule,Mass,Hernia,full_path,target_vector
0,00000001_000.png,Cardiomegaly,0,1,57,M,PA,2682,2749,0.143,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,../dataset/NIH-CXR/all_images/00000001_000.png,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,../dataset/NIH-CXR/all_images/00000001_001.png,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,../dataset/NIH-CXR/all_images/00000001_002.png,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
3,00000002_000.png,No Finding,0,2,80,M,PA,2500,2048,0.171,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,../dataset/NIH-CXR/all_images/00000002_000.png,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,00000003_001.png,Hernia,0,3,74,F,PA,2500,2048,0.168,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,../dataset/NIH-CXR/all_images/00000003_001.png,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,00000020_000.png,Pleural_Thickening,2,20,53,M,PA,2978,2991,0.143,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,../dataset/NIH-CXR/all_images/00000020_000.png,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
96,00000021_000.png,Nodule,0,21,77,M,PA,2500,2048,0.168,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,../dataset/NIH-CXR/all_images/00000021_000.png,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
97,00000021_001.png,Infiltration,1,21,80,M,PA,3056,2496,0.139,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,../dataset/NIH-CXR/all_images/00000021_001.png,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
98,00000022_000.png,No Finding,0,22,47,M,PA,2048,2500,0.171,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,../dataset/NIH-CXR/all_images/00000022_000.png,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [7]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(data, test_size = 0.2, random_state = 1996)
print('training set - # of observations: ', len(train_set))
print('test set - # of observations): ', len(test_set))
print('prior, full data set - # of observations): ', len(data))

training set - # of observations:  89696
test set - # of observations):  22424
prior, full data set - # of observations):  112120


In [8]:
phase = 'test'   ## select train or test
if phase == 'test':
    vit = FeatureSelectModule(image_size=224, input_size=768, num_classes=len(dummy_labels), hidden_size=256, C=0.5)
    pretrained_dict = torch.load('./train_model/NIH_epoch_199_no_FA.pth')
    model_dict = vit.state_dict()
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict and v.size() == model_dict[k].size()}
    model_dict.update(pretrained_dict)
    vit.load_state_dict(model_dict)
elif phase == 'train':
    vit = FeatureSelectModule(image_size=224, input_size=768, num_classes=15, hidden_size=256, C=0.5)

Resized positional embeddings from torch.Size([1, 577, 768]) to torch.Size([1, 197, 768])
Loaded pretrained weights.


In [9]:
# cal mutil class loss
class FocalLoss(nn.Module):
    def __init__(self, device, gamma = 1.0):
        super(FocalLoss, self).__init__()
        self.device = device
        self.gamma = torch.tensor(gamma, dtype = torch.float32).to(device)
        self.eps = 1e-6        
    def forward(self, input, target):
        BCE_loss = F.binary_cross_entropy_with_logits(input, target, reduction='none').to(self.device)
        pt = torch.exp(-BCE_loss) # prevents nans when probability 0
        F_loss = (1-pt)**self.gamma * BCE_loss
        return F_loss.mean()*10
     

## calc auc
from sklearn.metrics import roc_auc_score
def cal_auc(label, predict):
    class_roc_auc_list = []
    useful_classes_roc_auc_list = []
    for i in range(label.shape[1]):
        class_roc_auc = roc_auc_score(label[:, i], predict[:, i])
        class_roc_auc_list.append(class_roc_auc)
        if i != 0:
            useful_classes_roc_auc_list.append(class_roc_auc)
    return np.mean(np.array(useful_classes_roc_auc_list))

In [10]:
base_lr = 0.0005
batch_size = 64
max_epoch = 200

vit = vit.to(device)
fn_loss  = FocalLoss(device = device, gamma = 2.).to(device)
optimizer = torch.optim.SGD(vit.parameters(), lr=base_lr)

train_NIH_data = NIH(data=train_set, phase = 'train', img_size=(224, 224))
train_data_loader = DataLoader(train_NIH_data, batch_size=batch_size, shuffle=True, num_workers=5)
test_NIH_data = NIH(data=test_set, phase = 'test', img_size=(224, 224))
test_data_loader = DataLoader(test_NIH_data, batch_size=batch_size, shuffle=True, num_workers=5)

In [11]:
if phase == 'train':
    import time
    iter_num = 0
    vit.train()
    for epoch_num in range(0, max_epoch):
        train_loader_nums = len(train_data_loader.dataset)
        probs = np.zeros((train_loader_nums, len(dummy_labels)), dtype = np.float32)
        gt    = np.zeros((train_loader_nums, len(dummy_labels)), dtype = np.float32)
        k=0

        start_time = time.time()
        for train_data_batch, train_labels_batch in train_data_loader:
            train_labels_batch = train_labels_batch.to(device)
            outputs, similiay_loss, contrast_loss, entrop_loss = vit(train_data_batch)
            outputs = outputs.reshape(outputs.shape[0], -1)           
            train_labels_batch = train_labels_batch.reshape(train_labels_batch.shape[0], -1)

            # storing model predictions for metric evaluat`ion 
            probs[k: k + outputs.shape[0], :] = outputs.cpu().detach().numpy()
            gt[   k: k + outputs.shape[0], :] = train_labels_batch.cpu().detach().numpy()
            k += outputs.shape[0]

            train_loss = fn_loss(outputs, train_labels_batch)
            train_loss += contrast_loss/5000
            train_loss -= similiay_loss/batch_size

            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()

        auc = cal_auc(gt, probs)
        print(f"epoch_num {epoch_num} train loss {train_loss} contrast loss {contrast_loss}  train auc {auc}")  
        end_time = time.time()  
        elapsed_time = end_time - start_time
        print(f"time consume ：{elapsed_time} s")

        lr_ = base_lr*(1-0.0009)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr_
        iter_num = iter_num + 1

        save_interval = 25  # int(max_epoch/6)
        if (epoch_num + 1) % save_interval == 0:
            save_mode_path = os.path.join('./train_model', 'NIH_epoch_' + str(epoch_num) + '.pth')
            torch.save(vit.state_dict(), save_mode_path)
            print("save model to {}".format(save_mode_path))

        if epoch_num >= max_epoch - 1:
            save_mode_path = os.path.join('./train_model', 'NIH_epoch_' + str(epoch_num) + '.pth')
            torch.save(vit.state_dict(), save_mode_path)
            print("save model to {}".format(save_mode_path))
            break

In [12]:
def entropy(logits):
    probabilities = F.softmax(logits, dim=1)
    H = -torch.sum(probabilities * torch.log(probabilities), dim=1)
    return H

In [13]:
total_entrpy = 0.0
#### eval model, don`t use FA-Loss
test_loader_nums = len(test_data_loader.dataset)
probs = np.zeros((test_loader_nums, 15), dtype = np.float32)
gt    = np.zeros((test_loader_nums, 15), dtype = np.float32)
k=0

vit.eval()
with torch.no_grad():
    for test_data_batch, test_label_batch in test_data_loader:
        test_label_batch = test_label_batch.to("cuda:0")
        test_outputs, similiay_loss, contrast_loss, entrop_loss = vit(test_data_batch)
        test_outputs = test_outputs.reshape(test_outputs.shape[0], -1)   
        
        entr_loss = torch.sum(entropy(test_outputs))
        total_entrpy += entr_loss
        
        test_label_batch = test_label_batch.reshape(test_outputs.shape[0], -1)
        # storing model predictions for metric evaluat`ion 
        probs[k: k + test_outputs.shape[0], :] = test_outputs.cpu().detach().numpy()
        gt[   k: k + test_outputs.shape[0], :] = test_label_batch.cpu().detach().numpy()
        k += test_outputs.shape[0]
    print(f"##### total enp {total_entrpy}")
    print(cal_auc(gt, probs))

##### total enp 51812.48046875
0.7706467536518739


epoch 200 100%sample 0.7805   using FA-loss